# Votre premier réseau de neurones

Dans ce projet, vous allez construire votre premier réseau neuronal.
En utilisant le jeu de données et vous l'utiliserez pour prédire le nombre de personnes qui louent des vélos chaque jour.

Nous avons fourni une partie du code, mais nous vous avons laissé le soin de mettre en œuvre le réseau neuronal (pour l'essentiel).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# A executer si code exécuté dans Collab
from google.colab import drive
drive.mount('/content/drive')

#◢ Chargement des données

In [ ]:
exercice1_path = 'drive/My Drive/DeepLearning/Exercice_01_Premier_reseau'
data_path = exercice1_path + '/Bike-Sharing-Dataset/hour.csv'

rides = pd.read_csv(data_path)

In [ ]:
rides.head()

#◢ Préparation des données

Une étape cruciale dans le travail en machine learning est la préparation des données. 

Ce jeu de données contient le nombre total de vélos loués pour chaque heure de chaque jour du 1er Janvier 2011 au 31 Décembre 2012. Il contient également le nombre de vélos loués par les abonnés et les utilisateurs occasionnels du système.


Le fichier hour.csv contient les informations suivantes :
- instant: record index
- dteday : date
- season : season (1:springer, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

Le graphique ci-dessous montre le nombre de cyclistes au cours des 10 premiers jours environ de l'ensemble des données. (Certaines données sont manquantes et certains jours n'ont pas exactement 24 entrées dans le jeu de données, ce n'est donc pas exactement 10 jours). 

Vous pouvez voir les locations réparties par heure ici. 

Ces données sont assez compliquées ! 

Les week-ends ont des taux de fréquentation plus bas que les autres jours et il y a des pics lorsque les gens vont et viennent à vélo au travail pendant la semaine. 
En examinant les données ci-dessus, nous avons également des informations sur la température, l'humidité et la vitesse du vent, qui ont toutes une incidence sur le nombre de vélos loués. 

Vous allez essayer de saisir tout cela avec votre modèle.


In [ ]:
rides[:24*10].plot(x='dteday', y='cnt', figsize=(20, 10))

## Dummification des variables

Le jeu de données contient quelques variables catégorielles comme la saison, le temps, le mois.

Pour les inclure dans notre modèle, nous devons les convertir en variables fictives binaires. 

C'est simple à faire avec la librairie pandas grâce à `get_dummies()`.

In [ ]:
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)
data.head()

## Standardisation des variables cibles
Pour faciliter l'apprentissage du réseau de neurones et la création d'un modèle performant, nous allons standardiser chacune des variables continues. Autrement dit, nous allons déplacer et mettre à l'échelle les variables de telle sorte qu'elles aient une moyenne nulle et un écart-type de 1.

Les facteurs d'échelle sont enregistrés afin que nous puissions revenir en arrière lorsque nous utiliserons le réseau pour les prédictions.

In [ ]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

## Diviser les données en 3 jeux de données : entrainement, validation et test

Les données des 21 derniers jours (environ) seront utilisées pour le jeu de test.

In [ ]:
# Save data for approximately the last 21 days 
test_data = data[-21*24:]

# Now remove the test data from the data set 
data = data[:-21*24]

# Separate the data into features and targets
target_fields = ['cnt', 'casual', 'registered']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

Comme il s'agit de données chronologiques (time series), nous séparons le jeu de données en deux parties en gardant la notion temporelle.
Les données les plus anciennes constitueront le jeu d'entrainement et les 60 derniers jours constitueront le jeu de données de validation.

In [ ]:
# Hold out the last 60 days or so of the remaining data as a validation set
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

#◢ Construction du réseau

Ci-dessous, vous trouverez la structure du code et les tests unitaires permettant de vérifier que votre implémentation est correcte.

Vous allez mettre en place la propagation avant et arrière du réseau. 
Vous définirez également les hyperparamètres : le taux d'apprentissage, le nombre de neurones cachés et le nombre de fois où l'algorithme parcourira l'intégralité du jeu de données d'entrainement.

<img class="tfo-display-only-on-site" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Artificial_neural_network.svg/1147px-Artificial_neural_network.svg.png" height="300"/>

Le réseau comporte deux couches, une couche cachée et une couche de sortie. 
La couche cachée utilisera la fonction sigmoïde pour les activations. 
La couche de sortie n'a qu'un seul neurone et est utilisée pour la régression, la sortie du neurone est la même que l'entrée du neurone. C'est-à-dire que la fonction d'activation est $f(x)=x$. 

Nous utilisons les poids pour propager les signaux vers l'avant des couches d'entrée aux couches de sortie du réseau de neurones. Nous utilisons également les poids pour propager l'erreur en arrière de la sortie vers le réseau afin de mettre à jour nos poids. C'est ce qu'on appelle la *propagation arrière* ou *back propagation*.

> **ASTUCE** Vous aurez besoin de la dérivée de la fonction d'activation de la sortie ($f(x) = x$) pour implémentater la backpropagation. Si vous n'êtes pas familier avec le calcul, cette fonction est équivalente à l'équation $y = x$. Quelle est la pente de cette équation ? C'est la dérivée de $f(x)$.

Ci-dessous, les taches à implémenter :
1. Implémenter la fonction sigmoïde à utiliser comme fonction d'activation. Définissez `self.activation_function` dans `__init__`.
2. Implémenter la propagation avant dans la méthode `forwardPass`.
3. Implémenter le calcul de l'erreur de sortie dans la méthode `train`.


In [ ]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        self.lr = learning_rate
        
        #### TODO: Set self.activation_function to your implemented sigmoid function ####
        #
        # Note: in Python, you can define a function with a lambda expression,
        # as shown below.
        self.activation_function = 0  # Replace 0 with your sigmoid calculation.
        
        ### If the lambda code above is not something you're familiar with,
        # You can uncomment out the following three lines and put your 
        # implementation there instead.
        #
        #def sigmoid(x):
        #    return 0  # Replace 0 with your sigmoid calculation here
        #self.activation_function = sigmoid


    def forwardPass(self, X):
        ''' Run a forward pass through the network with input features 
        
            Arguments
            ---------
            features: 1D array of feature values
        '''
        #### Implement the forward pass here ####
        # TODO: Hidden layer - replace these values with the appropriate calculations.
        hidden_inputs = XXX # signals into hidden layer
        hidden_outputs = XXX # signals from hidden layer
        
        # TODO: Output layer - Replace these values with the appropriate calculations.
        final_inputs = XXX # signals into final output layer
        final_outputs = XXX # signals from final output layer 
        
        return hidden_outputs, final_outputs

    def backwardPass(self, X, delta_weights_i_h, delta_weights_h_o, hidden_outputs, error):
        ''' Run a backward pass through the network from error output

            https://en.wikipedia.org/wiki/Delta_rule
            
            Arguments
            ---------
            X: inputs
            delta_weights_i_h:
            delta_weights_h_o:
            hidden_outputs:
            error:
        '''
        # Calculate the hidden layer's contribution to the error
        hidden_error = error * self.weights_hidden_to_output.T
        
        # Backpropagated error terms
        # Gradient =  Learning Rate * Erreur * Dérivée de la sortie
        output_error_term = self.lr * error
        hidden_error_term = self.lr * hidden_error * hidden_outputs * (1.0 - hidden_outputs)

        #  DeltaPoids = Gradient (Produit matriciel) Input
        # Weight step (input to hidden)
        delta_weights_i_h += np.dot(X[:,None],hidden_error_term)
        # Weight step (hidden to output)
        delta_weights_h_o += np.dot(hidden_outputs[:,None],output_error_term[:,None])

        return delta_weights_i_h, delta_weights_h_o
    
    def train(self, features, targets):
        ''' Train the network on batch of features and targets. 
        
            Arguments
            ---------
            
            features: 2D array, each row is one data record, each column is a feature
            targets: 1D array of target values
        
        '''
        n_records = features.shape[0]
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
        for X, y in zip(features, targets):
            
            ### Forward pass ###
            hidden_outputs, final_outputs = self.forwardPass(X)

            # TODO: Output error - Replace this value with your calculations.
            error = XXX # Output layer error is the difference between desired target and actual output.

            ### Backward pass ###
            delta_weights_i_h, delta_weights_h_o = self.backwardPass(X, delta_weights_i_h, delta_weights_h_o, hidden_outputs, error)
            

        # Update the weights - Replace these values with your calculations.
        self.weights_hidden_to_output +=  delta_weights_h_o / n_records # update hidden-to-output weights with gradient descent step
        self.weights_input_to_hidden +=  delta_weights_i_h / n_records # update input-to-hidden weights with gradient descent step

#◢ Tests unitaires

Run these unit tests to check the correctness of your network implementation. This will help you be sure your network was implemented correctly befor you starting trying to train it. These tests must all be successful to pass the project.

Lancez ces tests unitaires pour vérifier l'exactitude du fonctionnement de votre réseau. Cela vous permettra de vous assurer que votre réseau a été correctement implémenter avant de commencer à l'entrainer. 

Bien entendu, tous ces tests doivent tous être réussis !

In [ ]:
import unittest

inputs = np.array([[0.5, -0.2, 0.1]])
targets = np.array([[0.4]])
test_w_i_h = np.array([[0.1, -0.2],
                       [0.4, 0.5],
                       [-0.3, 0.2]])
test_w_h_o = np.array([[0.3],
                       [-0.1]])

class TestMethods(unittest.TestCase):
    
    ##########
    # Unit tests for data loading
    ##########
    
    def test_data_path(self):
        # Test that file path to dataset has been unaltered
        self.assertTrue(data_path.lower() == exercice1_path.lower() + '/bike-sharing-dataset/hour.csv')
        
    def test_data_loaded(self):
        # Test that data frame loaded
        self.assertTrue(isinstance(rides, pd.DataFrame))
    
    ##########
    # Unit tests for network functionality
    ##########

    def test_activation(self):
        network = NeuralNetwork(3, 2, 1, 0.5)
        # Test that the activation function is a sigmoid
        self.assertTrue(np.all(network.activation_function(0.5) == 1/(1+np.exp(-0.5))))

    def test_train(self):
        # Test that weights are updated correctly on training
        network = NeuralNetwork(3, 2, 1, 0.5)
        network.weights_input_to_hidden = test_w_i_h.copy()
        network.weights_hidden_to_output = test_w_h_o.copy()
        
        network.train(inputs, targets)
        self.assertTrue(np.allclose(network.weights_hidden_to_output, 
                                    np.array([[ 0.37275328], 
                                              [-0.03172939]])))
        self.assertTrue(np.allclose(network.weights_input_to_hidden,
                                    np.array([[ 0.10562014, -0.20185996], 
                                              [0.39775194, 0.50074398], 
                                              [-0.29887597, 0.19962801]])))

    def test_forwardPass(self):
        # Test correctness of forwardPass method
        network = NeuralNetwork(3, 2, 1, 0.5)
        network.weights_input_to_hidden = test_w_i_h.copy()
        network.weights_hidden_to_output = test_w_h_o.copy()
        hidden_outputs, final_outputs = network.forwardPass(inputs)

        self.assertTrue(np.allclose(final_outputs, 0.09998924))

suite = unittest.TestLoader().loadTestsFromModule(TestMethods())
unittest.TextTestRunner().run(suite)

#◢ Entrainement du réseau

Ici, vous allez définir les hyperparamètres du réseau. 
La stratégie consiste à trouver les bons hyperparamètres de telle sorte que l'erreur de prédiction sur le jeu d'entraînement soit la plus faible possible, en faisant attention de ne pas tomber dans l'overfitting. 
En effet, si vous entraînez le réseau trop longtemps ou si vous avez trop de neurones cachés, il peut devenir trop spécifique au jeu d'entraînement et ne pourra pas généraliser son apprentissage à d'autres données. Autrement dit, faites attention dès lors que l'erreur sur le jeu de validation commencera à augmenter à mesure que l'erreur sur le jeu de données d'entrainement diminuera.

Vous utiliserez également une méthode appelée "Stochastic Gradient Descent" (SGD) pour entrainer le réseau. L'idée est que pour chaque itération, vous prenez un échantillon aléatoire des données au lieu d'utiliser l'ensemble des données. Vous effectuerez ainsi beaucoup plus d'itérations qu'avec une descente normale de gradient, mais chaque itération sera beaucoup plus rapide. Cela permet d'entraîner le réseau plus efficacement. Vous en saurez plus sur le SGD plus tard.

### Choisissez le nombre d'itérations (epoch)
Cycle d'apprentissage complet sur l'intégralité de l'ensemble de données de manière à ce que chaque exemple ait été vu une fois. Une itération représente ainsi N/taille du lot itérations d'apprentissage (batch), où N est le nombre total d'exemples. Plus vous utiliserez d'itérations, plus le modèle sera adapté aux données. Cependant, si vous utilisez trop d'itérations, alors le modèle ne s'adaptera pas bien aux autres données, c'est ce qu'on appelle le surajustement ou overfitting.

### Choisissez le taux d'apprentissage (learning rate)
Grandeur scalaire utilisée pour entraîner le modèle via la descente de gradient. À chaque itération, l'algorithme de descente de gradient multiplie le taux d'apprentissage par le gradient. Le produit ainsi généré est appelé pas de gradient.
Si le pas est trop grand, les poids ont tendance à exploser et le réseau ne parvient pas à ajuster les données. Un bon choix pour commencer est de commencer à 0,1. Si le réseau a des difficultés à ajuster les données, essayez de réduire le taux d'apprentissage. Notez que plus le taux d'apprentissage est faible, plus les étapes de la mise à jour des poids sont petites et plus le réseau neuronal met du temps à converger.

### Choisissez le nombre de neurones cachés
Plus vous avez de neurones cachés, plus les prédictions du modèle seront précises. Essayez quelques chiffres différents et voyez comment cela affecte la performance. Vous pouvez consulter le dictionnaire des pertes pour obtenir une mesure de la performance du réseau. Si le nombre de neurones cachées est trop faible, le modèle n'aura pas assez d'espace pour apprendre et s'il est trop élevé, il y a trop d'options quant à la direction que peut prendre l'apprentissage. L'astuce consiste ici à trouver le bon équilibre dans le nombre de neurones cachés que vous choisissez.

In [ ]:
import sys

### Set the hyperparameters here ###
iterations = 1000
learning_rate = 0.8
hidden_nodes = 10
output_nodes = 1

def MSE(y, Y):
    return np.mean((y-Y)**2)

N_i = train_features.shape[1]
network = NeuralNetwork(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for ii in range(iterations):
    # Go through a random batch of 128 records from the training data set
    batch = np.random.choice(train_features.index, size=128)
    X, y = train_features.iloc[batch].values, train_targets.iloc[batch]['cnt']
                             
    network.train(X, y)
    
    # Printing out the training progress
    train_loss = MSE(network.forwardPass(train_features)[1].T, train_targets['cnt'].values)
    val_loss = MSE(network.forwardPass(val_features)[1].T, val_targets['cnt'].values)
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations)) \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    sys.stdout.flush()
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(losses['train'], label='Training loss')
plt.plot(losses['validation'], label='Validation loss')
plt.legend()
_ = plt.ylim()

#◢ Vérification de vos prédictions

Ici, utilisez les données de test pour voir dans quelle mesure votre réseau est performant. 
Si quelque chose ne va pas, assurez-vous que chaque étape de votre réseau est correctement mise en œuvre.

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

mean, std = scaled_features['cnt']
predictions = network.forwardPass(test_features)[1].T*std + mean
ax.plot(predictions[0], label='Prediction')
ax.plot((test_targets['cnt']*std + mean).values, label='Data')
ax.set_xlim(right=len(predictions))
ax.legend()

dates = pd.to_datetime(rides.iloc[test_data.index]['dteday'])
dates = dates.apply(lambda d: d.strftime('%b %d'))
ax.set_xticks(np.arange(len(dates))[12::24])
_ = ax.set_xticklabels(dates[12::24], rotation=45)